In [5]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
import tensorflow as tf
import datetime

In [6]:
mnist = tf.keras.datasets.mnist

In [9]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [18]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
      ])

In [19]:
model = create_model()

In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #loga폴더안에 fit폴더안에 현재시간자동화(윈도우에서는/를 \\로 표시한다.)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [25]:
model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.2189 - accuracy: 0.9352 - val_loss: 0.1113 - val_accuracy: 0.9660
Epoch 2/5
60000/60000 [==============================] - 4s 68us/sample - loss: 0.0955 - accuracy: 0.9708 - val_loss: 0.0785 - val_accuracy: 0.9759
Epoch 3/5
60000/60000 [==============================] - 4s 69us/sample - loss: 0.0692 - accuracy: 0.9781 - val_loss: 0.0710 - val_accuracy: 0.9784
Epoch 4/5
60000/60000 [==============================] - 4s 66us/sample - loss: 0.0543 - accuracy: 0.9829 - val_loss: 0.0668 - val_accuracy: 0.9787
Epoch 5/5
60000/60000 [==============================] - 4s 62us/sample - loss: 0.0426 - accuracy: 0.9860 - val_loss: 0.0707 - val_accuracy: 0.9784


In [16]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %tensorboard  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%p

In [28]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 10644), started 0:19:31 ago. (Use '!kill 10644' to kill it.)

In [33]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [34]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [35]:
# Define our metrics
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [39]:
def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [44]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logss\\gradient_tape\\' + current_time + '\\train'
test_log_dir = 'logss\\gradient_tape\\' + current_time + '\\test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [45]:
%tensorboard --logdir logs/gradient_tape

Reusing TensorBoard on port 6006 (pid 2800), started 0:01:44 ago. (Use '!kill 2800' to kill it.)